In [23]:
import warnings
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
import mne
import moabb
from moabb.datasets import BNCI2014_001
from moabb.evaluations import WithinSessionEvaluation
from moabb.paradigms import LeftRightImagery
from mne.decoding import CSP
from mne import Epochs, pick_types, events_from_annotations
from pykan.kan import KAN
from sklearn.metrics import f1_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from torch.nn import Sequential, Linear, ReLU, CrossEntropyLoss, Module
from pykan.kan import LBFGS 
from torch.utils.data import DataLoader, TensorDataset
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')# if torch.cuda.is_available() else 'cpu')

import torch.optim as optim
from tqdm import tqdm
import time
import json


epochs=20
batch = 16
lr = 0.1
loss_fn = CrossEntropyLoss()
steps=50
kf = KFold(n_splits=10, shuffle=True, random_state=42)  
subjects= [8]
le = LabelEncoder()
hidden_layers= 10

moabb.set_log_level("info")
warnings.filterwarnings("ignore")
csp = CSP(n_components=4, reg='oas') 


In [2]:
def data_EEG(subjects, train=True, multiclass=False):
        if train:
                session_name = "0train"

        else:
                session_name="1test"

        
        dataset = BNCI2014_001()
        if multiclass:
                event_id = dict(left=1, right=2, feet=3, tongue=4)
        else:
                event_id = dict(left=1, right=2)#, feet=3, tongue=4)

        tmin, tmax = -1., 6.
        runs = range(5)

        epochs_data=[]
        for subject in subjects:
                sessions = dataset.get_data(subjects=[subject])
                for run in runs:
                        raw = sessions[subject][session_name][str(run)]
                        events = mne.find_events(raw, stim_channel="stim")        
                        picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
                        raw.filter(7., 35., method='iir', picks=picks)
                        
                        epochs = Epochs(raw,
                                events,
                                event_id,
                                tmin,
                                tmax,
                                proj=True,
                                picks=picks,
                                baseline=None,
                                preload=True,
                                verbose=False)
                        epochs_data.append(epochs)
        return epochs_data

def export_json(data, filename):
    for key, value in data.items():
        if isinstance(value, np.ndarray):
            data[key] = value.tolist()
        elif isinstance(value, list) and all(isinstance(x, list) for x in value):  # Check if ALL items are lists
            data[key] = [[float(item) for item in sublist] for sublist in value]  # Convert inner elements to float
        elif isinstance(value, list): 
            data[key] = [float(item) for item in value]  # Convert the numbers to float in the list 


    with open(filename, "w") as outfile:
        json.dump(data, outfile, indent=4)


In [3]:
def creating_models(input_shape, output_shape):
    kan_model = KAN(width=[input_shape, output_shape], grid=3, k=3, 
    seed=0, device=device)         

    for param in kan_model.parameters():
        param.data = param.data.double()
        
    return kan_model

class MLP(Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.layers = Sequential(
            Linear(input_dim, hidden_dim),
            ReLU(),  # Activation function (you can experiment with others)
            Linear(hidden_dim, output_dim)  # Output layer
        )

    def forward(self, x):
        return self.layers(x)


    def train(self, dataset, opt="LBFGS", steps=100, log=1, lamb=0., lamb_l1=1., lamb_entropy=2., lamb_coef=0., lamb_coefdiff=0., update_grid=True, grid_update_num=10, loss_fn=None, lr=1., stop_grid_update_step=50, batch=-1,
              small_mag_threshold=1e-16, small_reg_factor=1., metrics=None, sglr_avoid=False, save_fig=False, in_vars=None, out_vars=None, beta=3, save_fig_freq=1, img_folder='./video', device='cpu'):
        '''
        training

        Args:
        -----
            dataset : dic
                contains dataset['train_input'], dataset['train_label'], dataset['test_input'], dataset['test_label']
            opt : str
                "LBFGS" or "Adam"
            steps : int
                training steps
            log : int
                logging frequency
            lamb : float
                overall penalty strength
            lamb_l1 : float
                l1 penalty strength
            lamb_entropy : float
                entropy penalty strength
            lamb_coef : float
                coefficient magnitude penalty strength
            lamb_coefdiff : float
                difference of nearby coefficits (smoothness) penalty strength
            update_grid : bool
                If True, update grid regularly before stop_grid_update_step
            grid_update_num : int
                the number of grid updates before stop_grid_update_step
            stop_grid_update_step : int
                no grid updates after this training step
            batch : int
                batch size, if -1 then full.
            small_mag_threshold : float
                threshold to determine large or small numbers (may want to apply larger penalty to smaller numbers)
            small_reg_factor : float
                penalty strength applied to small factors relative to large factos
            device : str
                device   
            save_fig_freq : int
                save figure every (save_fig_freq) step

        Returns:
        --------
            results : dic
                results['train_loss'], 1D array of training losses (RMSE)
                results['test_loss'], 1D array of test losses (RMSE)
                results['reg'], 1D array of regularization

        Example
        -------
        >>> # for interactive examples, please see demos
        >>> from utils import create_dataset
        >>> model = KAN(width=[2,5,1], grid=5, k=3, noise_scale=0.1, seed=0)
        >>> f = lambda x: torch.exp(torch.sin(torch.pi*x[:,[0]]) + x[:,[1]]**2)
        >>> dataset = create_dataset(f, n_var=2)
        >>> model.train(dataset, opt='LBFGS', steps=50, lamb=0.01);
        >>> model.plot()
        '''


        pbar = tqdm(range(steps), desc='description', ncols=100)
  
        if loss_fn == None:
            loss_fn = loss_fn_eval = lambda x, y: torch.mean((x - y) ** 2)
        else:
            loss_fn = loss_fn_eval = loss_fn

        acc = acc_eval = lambda x,y: torch.mean((torch.argmax(x, dim=1) == y).float())

        grid_update_freq = int(stop_grid_update_step / grid_update_num)

        if opt == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        elif opt == "LBFGS":
            optimizer = LBFGS(self.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe", tolerance_grad=1e-32, tolerance_change=1e-32, tolerance_ys=1e-32)

        results = {}
        results['train_loss'] = []
        results['test_loss'] = []
        results['train_acc'] = []
        results['test_acc'] = []
        if metrics != None:
            for i in range(len(metrics)):
                results[metrics[i].__name__] = []

        if batch == -1 or batch > dataset['train_input'].shape[0]:
            batch_size = dataset['train_input'].shape[0]
            batch_size_test = dataset['test_input'].shape[0]
        else:
            batch_size = batch
            batch_size_test = batch

        global train_loss, train_acc

        def closure():
            global train_loss, train_acc
            optimizer.zero_grad()
            pred = self.forward(dataset['train_input'][train_id].to(device))
            if sglr_avoid == True:
                id_ = torch.where(torch.isnan(torch.sum(pred, dim=1)) == False)[0]
                train_loss = loss_fn(pred[id_], dataset['train_label'][train_id][id_].to(device))
            else:
                train_loss = loss_fn(pred, dataset['train_label'][train_id].to(device))
            train_acc = acc(pred, dataset['train_label'][train_id].to(device))

            objective = train_loss + lamb
            objective.backward()
            return objective

        if save_fig:
            if not os.path.exists(img_folder):
                os.makedirs(img_folder)
        
        for _ in pbar:

            train_id = np.random.choice(dataset['train_input'].shape[0], batch_size, replace=False)
            test_id = np.random.choice(dataset['test_input'].shape[0], batch_size_test, replace=False)

            #if _ % grid_update_freq == 0 and _ < stop_grid_update_step and update_grid:
            #    self.update_grid_from_samples(dataset['train_input'][train_id].to(device))

            if opt == "LBFGS":
                optimizer.step(closure)

            if opt == "Adam":
                pred = self.forward(dataset['train_input'][train_id].to(device))
                if sglr_avoid == True:
                    id_ = torch.where(torch.isnan(torch.sum(pred, dim=1)) == False)[0]
                    train_loss = loss_fn(pred[id_], dataset['train_label'][train_id][id_].to(device))
                else:
                    train_loss = loss_fn(pred, dataset['train_label'][train_id].to(device))
                loss = train_loss + lamb
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            test_loss = loss_fn_eval(self.forward(dataset['test_input'][test_id].to(device)), dataset['test_label'][test_id].to(device))
            test_acc = acc_eval(self.forward(dataset['test_input'][test_id].to(device)), dataset['test_label'][test_id].to(device))

            if _ % log == 0:
                pbar.set_description("train acc: %.3f | test acc: %.3f " % ((train_acc).cpu().detach().numpy(), (test_acc).cpu().detach().numpy()))

            if metrics != None:
                for i in range(len(metrics)):
                    results[metrics[i].__name__].append(metrics[i]().item())

            results['train_loss'].append(torch.sqrt(train_loss).cpu().detach().numpy())
            results['test_loss'].append(torch.sqrt(test_loss).cpu().detach().numpy())
            results['train_acc'].append(train_acc.cpu().detach().numpy())
            results['test_acc'].append(test_acc.cpu().detach().numpy())

            if save_fig and _ % save_fig_freq == 0:
                self.plot(folder=img_folder, in_vars=in_vars, out_vars=out_vars, title="Step {}".format(_), beta=beta)
                plt.savefig(img_folder + '/' + str(_) + '.jpg', bbox_inches='tight', dpi=200)
                plt.close()

        return results


<h1> TWO CLASS - RAW </h1>

In [5]:
epochs_EEG=mne.concatenate_epochs(data_EEG(subjects=subjects))
df = epochs_EEG.to_data_frame().drop(columns=['time','epoch'])
X = df.drop(columns=['condition']).values
y = df.condition.values
y = le.fit_transform(y)

X = torch.from_numpy(X).double()
y  = torch.from_numpy(y).long()


input_layer = X.shape[1]
output_layer = len(np.unique(y))


kan_model  = creating_models(input_layer, output_layer)
kan_optimizer = LBFGS(kan_model.parameters(), lr=lr) 

mlp_model = MLP(input_layer, hidden_layers, output_layer).to(device)

for param in mlp_model.parameters():
    param.data = param.data.double()
mlp_optimizer = LBFGS(mlp_model.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe", tolerance_grad=1e-32, tolerance_change=1e-32, tolerance_ys=1e-32) #(mlp_model.parameters(), lr=lr)


metrics_KAN = {}  
metrics_KAN['train'] = []
metrics_KAN['test'] = []

metrics_MLP = {}  
metrics_MLP['train'] = []
metrics_MLP['test'] = []

total_train_time_KAN = 0
total_train_time_MLP = 0

for fold, (train_index, test_index) in enumerate(kf.split(X)):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    dataset={}
    dataset['train_input']= X_train
    dataset['train_label']= y_train
    dataset['test_input']= X_test
    dataset['test_label']= y_test

    fold_metrics_KAN_train = []
    fold_metrics_KAN_test =[]
    fold_metrics_MLP_train = []
    fold_metrics_MLP_test =[]

    for epoch in range(epochs):
        print('###### EPOCH ', epoch, " ############")

        epoch_start_time_KAN = time.time()

        results_kan = kan_model.train(dataset, opt="LBFGS", steps=steps, 
        loss_fn=loss_fn, batch=batch, lr=lr)
        
        epoch_end_time_KAN = time.time()
        epoch_train_KAN = epoch_end_time_KAN - epoch_start_time_KAN
        total_train_time_KAN += epoch_train_KAN
        
        epoch_start_time_MLP = time.time()

        fold_metrics_KAN_train.append(np.mean(results_kan['train_acc']))
        fold_metrics_KAN_test.append(np.mean(results_kan['test_acc'])) 

        results_mlp = mlp_model.train(dataset, opt="LBFGS", steps=steps, 
        loss_fn=loss_fn, batch=batch, lr=lr)

        epoch_end_time_MLP = time.time()
        epoch_train_MLP = epoch_end_time_MLP - epoch_start_time_MLP
        total_train_time_MLP += epoch_train_MLP
        
        fold_metrics_MLP_train.append(np.mean(results_mlp['train_acc']))
        fold_metrics_MLP_test.append(np.mean(results_mlp['test_acc'])) 

    metrics_KAN['train'].append(fold_metrics_KAN_train)
    metrics_KAN['test'].append(fold_metrics_KAN_test)
    
    metrics_MLP['train'].append(fold_metrics_MLP_train)
    metrics_MLP['test'].append(fold_metrics_MLP_test)

    
print(f"Training time KAN: {total_train_time_KAN:.2f} seconds")
print(f"Training time MLP: {total_train_time_MLP:.2f} seconds")
        

###### EPOCH  0  ############


train acc: 1.000 | test acc: 0.312 | reg: 14.121 : 100%|████████████| 50/50 [00:09<00:00,  5.32it/s]
train acc: 0.875 | test acc: 0.562 : 100%|██████████████████████████| 50/50 [00:00<00:00, 53.77it/s]


###### EPOCH  1  ############


train acc: 0.812 | test acc: 0.562 | reg: 22.644 : 100%|████████████| 50/50 [00:09<00:00,  5.32it/s]
train acc: 0.812 | test acc: 0.438 : 100%|██████████████████████████| 50/50 [00:00<00:00, 53.31it/s]


###### EPOCH  2  ############


...

###### EPOCH  18  ############


train acc: 0.625 | test acc: 0.625 | reg: 11.413 : 100%|████████████| 50/50 [00:14<00:00,  3.41it/s]
train acc: 0.625 | test acc: 0.375 : 100%|██████████████████████████| 50/50 [00:01<00:00, 41.64it/s]


###### EPOCH  19  ############


train acc: 0.875 | test acc: 0.750 | reg: 13.862 : 100%|████████████| 50/50 [00:13<00:00,  3.70it/s]
train acc: 0.750 | test acc: 0.375 : 100%|██████████████████████████| 50/50 [00:01<00:00, 41.22it/s]

Training time KAN: 2016.92 seconds
Training time MLP: 198.72 seconds


In [5]:
epochs_EEG=mne.concatenate_epochs(data_EEG(subjects=subjects, train=False))
df = epochs_EEG.to_data_frame().drop(columns=['time','epoch'])
X = df.drop(columns=['condition']).values
y = df.condition.values
y = le.fit_transform(y)
X = torch.from_numpy(X).double()
 
pred_start_time_KAN = time.time()
output_KAN= kan_model(X)
pred_time_KAN = time.time() - pred_start_time_KAN


pred_start_time_MLP = time.time()
output_MLP= mlp_model(X)
pred_time_MLP = time.time() - pred_start_time_KAN

metrics_KAN['pred']=torch.argmax(output_KAN, dim=1).numpy()
metrics_KAN['real']=y
metrics_KAN['time']=[total_train_time_KAN, pred_time_KAN]

filename = 'two_class_RAW_KAN_'+str(subjects[0])+".json"
export_json(metrics_KAN,filename)


metrics_MLP['pred']=torch.argmax(output_MLP, dim=1).numpy()
metrics_MLP['real']=y
metrics_MLP['time']=[float(total_train_time_MLP), float(pred_time_MLP)]

filename = 'two_class_RAW_MLP_'+str(subjects[0])+".json"
export_json(metrics_MLP,filename)

<h1> MULTICLASS - RAW </h1>

In [17]:
epochs_EEG=mne.concatenate_epochs(data_EEG(subjects=subjects, multiclass=True))
df = epochs_EEG.to_data_frame().drop(columns=['time','epoch'])
X = df.drop(columns=['condition']).values
y = df.condition.values
y = le.fit_transform(y)

X = torch.from_numpy(X).double()
y  = torch.from_numpy(y).long()


input_layer = X.shape[1]
output_layer = len(np.unique(y))


kan_model  = creating_models(input_layer, output_layer)
kan_optimizer = LBFGS(kan_model.parameters(), lr=lr) 

mlp_model = MLP(input_layer, hidden_layers, output_layer).to(device)

for param in mlp_model.parameters():
    param.data = param.data.double()
mlp_optimizer = LBFGS(mlp_model.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe", tolerance_grad=1e-32, tolerance_change=1e-32, tolerance_ys=1e-32) #(mlp_model.parameters(), lr=lr)


metrics_KAN = {}  
metrics_KAN['train'] = []
metrics_KAN['test'] = []

metrics_MLP = {}  
metrics_MLP['train'] = []
metrics_MLP['test'] = []

total_train_time_KAN = 0
total_train_time_MLP = 0

for fold, (train_index, test_index) in enumerate(kf.split(X)):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    dataset={}
    dataset['train_input']= X_train
    dataset['train_label']= y_train
    dataset['test_input']= X_test
    dataset['test_label']= y_test

    fold_metrics_KAN_train = []
    fold_metrics_KAN_test =[]
    fold_metrics_MLP_train = []
    fold_metrics_MLP_test =[]

    for epoch in range(epochs):
        print('###### EPOCH ', epoch, " ############")

        epoch_start_time_KAN = time.time()

        results_kan = kan_model.train(dataset, opt="LBFGS", steps=steps, 
        loss_fn=loss_fn, batch=batch, lr=lr)
        
        epoch_end_time_KAN = time.time()
        epoch_train_KAN = epoch_end_time_KAN - epoch_start_time_KAN
        total_train_time_KAN += epoch_train_KAN
        
        epoch_start_time_MLP = time.time()

        fold_metrics_KAN_train.append(np.mean(results_kan['train_acc']))
        fold_metrics_KAN_test.append(np.mean(results_kan['test_acc'])) 

        results_mlp = mlp_model.train(dataset, opt="LBFGS", steps=steps, 
        loss_fn=loss_fn, batch=batch, lr=lr)

        epoch_end_time_MLP = time.time()
        epoch_train_MLP = epoch_end_time_MLP - epoch_start_time_MLP
        total_train_time_MLP += epoch_train_MLP
        
        fold_metrics_MLP_train.append(np.mean(results_mlp['train_acc']))
        fold_metrics_MLP_test.append(np.mean(results_mlp['test_acc'])) 

    metrics_KAN['train'].append(fold_metrics_KAN_train)
    metrics_KAN['test'].append(fold_metrics_KAN_test)
    
    metrics_MLP['train'].append(fold_metrics_MLP_train)
    metrics_MLP['test'].append(fold_metrics_MLP_test)

    
print(f"Training time KAN: {total_train_time_KAN:.2f} seconds")
print(f"Training time MLP: {total_train_time_MLP:.2f} seconds")
        

###### EPOCH  0  ############


train acc: 0.688 | test acc: 0.188 | reg: 14.578 : 100%|████████████| 50/50 [00:16<00:00,  3.12it/s]
train acc: 0.688 | test acc: 0.375 : 100%|██████████████████████████| 50/50 [00:01<00:00, 48.47it/s]


###### EPOCH  1  ############


train acc: 0.750 | test acc: 0.375 | reg: 16.132 : 100%|████████████| 50/50 [00:16<00:00,  3.12it/s]
train acc: 0.812 | test acc: 0.062 : 100%|██████████████████████████| 50/50 [00:01<00:00, 47.64it/s]


###### EPOCH  2  ############


...

###### EPOCH  18  ############


train acc: 0.438 | test acc: 0.250 | reg: 17.146 : 100%|████████████| 50/50 [00:16<00:00,  3.09it/s]
train acc: 0.375 | test acc: 0.250 : 100%|██████████████████████████| 50/50 [00:01<00:00, 46.39it/s]


###### EPOCH  19  ############


train acc: 0.625 | test acc: 0.125 | reg: 18.670 : 100%|████████████| 50/50 [00:16<00:00,  3.09it/s]
train acc: 0.625 | test acc: 0.375 : 100%|██████████████████████████| 50/50 [00:01<00:00, 46.74it/s]

Training time KAN: 3766.46 seconds
Training time MLP: 250.58 seconds


In [7]:
epochs_EEG=mne.concatenate_epochs(data_EEG(subjects=subjects, train=False, multiclass=True))
df = epochs_EEG.to_data_frame().drop(columns=['time','epoch'])
X = df.drop(columns=['condition']).values
y = df.condition.values
y = le.fit_transform(y)
X = torch.from_numpy(X).double()
 
pred_start_time_KAN = time.time()
output_KAN= kan_model(X)
pred_time_KAN = time.time() - pred_start_time_KAN


pred_start_time_MLP = time.time()
output_MLP= mlp_model(X)
pred_time_MLP = time.time() - pred_start_time_KAN

metrics_KAN['pred']=torch.argmax(output_KAN, dim=1).numpy()
metrics_KAN['real']=y
metrics_KAN['time']=[total_train_time_KAN, pred_time_KAN]

filename = 'multiclass_RAW_KAN_'+str(subjects[0])+".json"
export_json(metrics_KAN,filename)


metrics_MLP['pred']=torch.argmax(output_MLP, dim=1).numpy()
metrics_MLP['real']=y
metrics_MLP['time']=[float(total_train_time_MLP), float(pred_time_MLP)]

filename = 'multiclass_RAW_MLP_'+str(subjects[0])+".json"
export_json(metrics_MLP,filename)

<h1> TWO CLASS - CSP </h1> 

In [24]:
epochs_EEG=mne.concatenate_epochs(data_EEG(subjects=subjects, multiclass=False))

y = epochs_EEG.events[:, -1]
X = csp.fit_transform(1e6 *epochs_EEG.get_data(), y)  
y = le.fit_transform(y)

X = torch.from_numpy(X).double()
y  = torch.from_numpy(y).long()


input_layer = X.shape[1]
output_layer = len(np.unique(y))


kan_model  = creating_models(input_layer, output_layer)
kan_optimizer = LBFGS(kan_model.parameters(), lr=lr) 

mlp_model = MLP(input_layer, hidden_layers, output_layer).to(device)

for param in mlp_model.parameters():
    param.data = param.data.double()
mlp_optimizer = LBFGS(mlp_model.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe", tolerance_grad=1e-32, tolerance_change=1e-32, tolerance_ys=1e-32) #(mlp_model.parameters(), lr=lr)


metrics_KAN = {}  
metrics_KAN['train'] = []
metrics_KAN['test'] = []

metrics_MLP = {}  
metrics_MLP['train'] = []
metrics_MLP['test'] = []

total_train_time_KAN = 0
total_train_time_MLP = 0

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    # Prepare data for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    dataset={}
    dataset['train_input']= X_train
    dataset['train_label']= y_train
    dataset['test_input']= X_test
    dataset['test_label']= y_test

    fold_metrics_KAN_train = []
    fold_metrics_KAN_test =[]
    fold_metrics_MLP_train = []
    fold_metrics_MLP_test =[]

    for epoch in range(epochs):
        print('###### EPOCH ', epoch, " ############")

        epoch_start_time_KAN = time.time()

        results_kan = kan_model.train(dataset, opt="LBFGS", steps=steps, 
        loss_fn=loss_fn, batch=-1, lr=lr)
        
        epoch_end_time_KAN = time.time()
        epoch_train_KAN = epoch_end_time_KAN - epoch_start_time_KAN
        total_train_time_KAN += epoch_train_KAN
        
        epoch_start_time_MLP = time.time()

        fold_metrics_KAN_train.append(np.mean(results_kan['train_acc']))
        fold_metrics_KAN_test.append(np.mean(results_kan['test_acc'])) 

        results_mlp = mlp_model.train(dataset, opt="LBFGS", steps=steps, 
        loss_fn=loss_fn, batch=-1, lr=lr)

        epoch_end_time_MLP = time.time()
        epoch_train_MLP = epoch_end_time_MLP - epoch_start_time_MLP
        total_train_time_MLP += epoch_train_MLP
        
        fold_metrics_MLP_train.append(np.mean(results_mlp['train_acc']))
        fold_metrics_MLP_test.append(np.mean(results_mlp['test_acc'])) 

    metrics_KAN['train'].append(fold_metrics_KAN_train)
    metrics_KAN['test'].append(fold_metrics_KAN_test)
    
    metrics_MLP['train'].append(fold_metrics_MLP_train)
    metrics_MLP['test'].append(fold_metrics_MLP_test)

    
print(f"Training time KAN: {total_train_time_KAN:.2f} seconds")
print(f"Training time MLP: {total_train_time_MLP:.2f} seconds")
        

###### EPOCH  0  ############


train acc: 0.981 | test acc: 1.000 | reg: 166.931 : 100%|███████████| 50/50 [00:03<00:00, 12.76it/s]
train acc: 0.981 | test acc: 1.000 : 100%|█████████████████████████| 50/50 [00:00<00:00, 106.55it/s]


###### EPOCH  1  ############


train acc: 1.000 | test acc: 1.000 | reg: 399.788 : 100%|███████████| 50/50 [00:04<00:00, 12.34it/s]
train acc: 0.981 | test acc: 1.000 : 100%|█████████████████████████| 50/50 [00:00<00:00, 178.83it/s]


###### EPOCH  2  ############


...

###### EPOCH  18  ############


train acc: 1.000 | test acc: 0.917 | reg: 8518.921 : 100%|██████████| 50/50 [00:00<00:00, 64.08it/s]
train acc: 1.000 | test acc: 0.917 : 100%|█████████████████████████| 50/50 [00:00<00:00, 337.75it/s]


###### EPOCH  19  ############


train acc: 1.000 | test acc: 0.917 | reg: 8518.921 : 100%|██████████| 50/50 [00:00<00:00, 59.11it/s]
train acc: 1.000 | test acc: 0.917 : 100%|█████████████████████████| 50/50 [00:00<00:00, 341.21it/s]

Training time KAN: 206.77 seconds
Training time MLP: 50.72 seconds


In [9]:
epochs_EEG=mne.concatenate_epochs(data_EEG(subjects=subjects, train=False))
y = epochs_EEG.events[:, -1]
X = csp.fit_transform(1e6 *epochs_EEG.get_data(), y)  
y = le.fit_transform(y)
X = torch.from_numpy(X).double()
 
pred_start_time_KAN = time.time()
output_KAN= kan_model(X)
pred_time_KAN = time.time() - pred_start_time_KAN


pred_start_time_MLP = time.time()
output_MLP= mlp_model(X)
pred_time_MLP = time.time() - pred_start_time_KAN

metrics_KAN['pred']=torch.argmax(output_KAN, dim=1).numpy()
metrics_KAN['real']=y
metrics_KAN['time']=[total_train_time_KAN, pred_time_KAN]

filename = 'two_class_CSP_KAN_'+str(subjects[0])+".json"
export_json(metrics_KAN,filename)


metrics_MLP['pred']=torch.argmax(output_MLP, dim=1).numpy()
metrics_MLP['real']=y
metrics_MLP['time']=[float(total_train_time_MLP), float(pred_time_MLP)]

filename = 'two_class_CSP_MLP_'+str(subjects[0])+".json"
export_json(metrics_MLP,filename)

<h1>MULTICLASS CSP </h>

In [20]:
epochs_EEG=mne.concatenate_epochs(data_EEG(subjects=subjects, multiclass=True))

y = epochs_EEG.events[:, -1]
X = csp.fit_transform(1e6 *epochs_EEG.get_data(), y)  
y = le.fit_transform(y)

X = torch.from_numpy(X).double()
y  = torch.from_numpy(y).long()


input_layer = X.shape[1]
output_layer = len(np.unique(y))


kan_model  = creating_models(input_layer, output_layer)
kan_optimizer = LBFGS(kan_model.parameters(), lr=lr) 

mlp_model = MLP(input_layer, hidden_layers, output_layer).to(device)

for param in mlp_model.parameters():
    param.data = param.data.double()
mlp_optimizer = LBFGS(mlp_model.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe", tolerance_grad=1e-32, tolerance_change=1e-32, tolerance_ys=1e-32) #(mlp_model.parameters(), lr=lr)


metrics_KAN = {}  
metrics_KAN['train'] = []
metrics_KAN['test'] = []

metrics_MLP = {}  
metrics_MLP['train'] = []
metrics_MLP['test'] = []

total_train_time_KAN = 0
total_train_time_MLP = 0

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    # Prepare data for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    dataset={}
    dataset['train_input']= X_train
    dataset['train_label']= y_train
    dataset['test_input']= X_test
    dataset['test_label']= y_test

    fold_metrics_KAN_train = []
    fold_metrics_KAN_test =[]
    fold_metrics_MLP_train = []
    fold_metrics_MLP_test =[]

    for epoch in range(epochs):
        print('###### EPOCH ', epoch, " ############")

        epoch_start_time_KAN = time.time()

        results_kan = kan_model.train(dataset, opt="LBFGS", steps=steps, 
        loss_fn=loss_fn, batch=-1, lr=lr)
        
        epoch_end_time_KAN = time.time()
        epoch_train_KAN = epoch_end_time_KAN - epoch_start_time_KAN
        total_train_time_KAN += epoch_train_KAN
        
        epoch_start_time_MLP = time.time()

        fold_metrics_KAN_train.append(np.mean(results_kan['train_acc']))
        fold_metrics_KAN_test.append(np.mean(results_kan['test_acc'])) 

        results_mlp = mlp_model.train(dataset, opt="LBFGS", steps=steps, 
        loss_fn=loss_fn, batch=-1, lr=lr)

        epoch_end_time_MLP = time.time()
        epoch_train_MLP = epoch_end_time_MLP - epoch_start_time_MLP
        total_train_time_MLP += epoch_train_MLP
        
        fold_metrics_MLP_train.append(np.mean(results_mlp['train_acc']))
        fold_metrics_MLP_test.append(np.mean(results_mlp['test_acc'])) 

    metrics_KAN['train'].append(fold_metrics_KAN_train)
    metrics_KAN['test'].append(fold_metrics_KAN_test)
    
    metrics_MLP['train'].append(fold_metrics_MLP_train)
    metrics_MLP['test'].append(fold_metrics_MLP_test)

    
print(f"Training time KAN: {total_train_time_KAN:.2f} seconds")
print(f"Training time MLP: {total_train_time_MLP:.2f} seconds")
        

###### EPOCH  0  ############


train acc: 0.644 | test acc: 0.500 | reg: 53.662 : 100%|████████████| 50/50 [00:05<00:00,  8.62it/s]
train acc: 0.676 | test acc: 0.375 : 100%|██████████████████████████| 50/50 [00:00<00:00, 66.33it/s]


###### EPOCH  1  ############


train acc: 0.648 | test acc: 0.542 | reg: 75.457 : 100%|████████████| 50/50 [00:05<00:00,  8.71it/s]
train acc: 0.671 | test acc: 0.375 : 100%|█████████████████████████| 50/50 [00:00<00:00, 126.84it/s]


###### EPOCH  2  ############


...

###### EPOCH  18  ############


train acc: 0.694 | test acc: 0.458 | reg: 707.882 : 100%|███████████| 50/50 [00:08<00:00,  5.86it/s]
train acc: 0.657 | test acc: 0.583 : 100%|█████████████████████████| 50/50 [00:00<00:00, 118.81it/s]


###### EPOCH  19  ############


train acc: 0.694 | test acc: 0.458 | reg: 710.103 : 100%|███████████| 50/50 [00:08<00:00,  5.59it/s]
train acc: 0.657 | test acc: 0.583 : 100%|█████████████████████████| 50/50 [00:00<00:00, 118.10it/s]

Training time KAN: 1612.50 seconds
Training time MLP: 90.91 seconds


In [11]:
epochs_EEG=mne.concatenate_epochs(data_EEG(subjects=subjects, train=False, multiclass=True))
y = epochs_EEG.events[:, -1]
X = csp.fit_transform(1e6 *epochs_EEG.get_data(), y)  
y = le.fit_transform(y)
X = torch.from_numpy(X).double()
 
pred_start_time_KAN = time.time()
output_KAN= kan_model(X)
pred_time_KAN = time.time() - pred_start_time_KAN


pred_start_time_MLP = time.time()
output_MLP= mlp_model(X)
pred_time_MLP = time.time() - pred_start_time_KAN

metrics_KAN['pred']=torch.argmax(output_KAN, dim=1).numpy()
metrics_KAN['real']=y
metrics_KAN['time']=[total_train_time_KAN, pred_time_KAN]

filename = 'multiclass_CSP_KAN_'+str(subjects[0])+".json"
export_json(metrics_KAN,filename)


metrics_MLP['pred']=torch.argmax(output_MLP, dim=1).numpy()
metrics_MLP['real']=y
metrics_MLP['time']=[float(total_train_time_MLP), float(pred_time_MLP)]

filename = 'multiclass_CSP_MLP_'+str(subjects[0])+".json"
export_json(metrics_MLP,filename)

In [25]:
lib = ['x','x^2','x^3','x^4','exp','log','sqrt','tanh','sin','abs']
kan_model.auto_symbolic(lib=lib)
formulas=kan_model.symbolic_formula()[0]

formulas


fixing (0,0,0) with tanh, r2=0.9987830927394489
fixing (0,0,1) with tanh, r2=0.9987844456666457
fixing (0,1,0) with tanh, r2=0.9974044397868516
fixing (0,1,1) with tanh, r2=0.9978205290014331
fixing (0,2,0) with tanh, r2=0.9988928062233559
fixing (0,2,1) with tanh, r2=0.9994505995493326
fixing (0,3,0) with sin, r2=0.9964715019308482
fixing (0,3,1) with sin, r2=0.9964036477893259


[420.97*sin(4.86*x_4 + 1.23) - 16122.29*tanh(2.21*x_1 + 3.39) + 780.98*tanh(4.54*x_2 + 5.6) + 2676.55*tanh(4.3*x_3 + 4.89) + 12335.46,
 -432.92*sin(4.87*x_4 + 1.24) + 15501.89*tanh(2.2*x_1 + 3.37) - 694.05*tanh(4.49*x_2 + 5.52) - 3972.74*tanh(3.2*x_3 + 3.64) - 10812.41]

In [26]:
with open(f'twoclass_CSP.txt', 'w') as f:  
    for formula in formulas:
        f.write(str(formula))
        f.write('\n')

In [74]:
import sympy as sp

def acc_multiclass(formulas, X, y):
    batch = X.shape[0]
    correct = 0
    num_classes = len(formulas)
    for i in range(batch):
        logits = []
        for formula in formulas:
            logit = formula.subs({'x_1': X[i, 0], 'x_2': X[i, 1], 'x_3': X[i, 2],'x_4': X[i, 3]})
            logit = sp.re(logit) if sp.im(logit) != 0 else logit  # Take the real part if complex
            logits.append(float(logit.evalf())) 
        predicted_class = np.argmax(logits)
        correct += (predicted_class == y[i])
    return correct / batch

In [75]:
train_acc = acc_multiclass(formulas, dataset['train_input'], dataset['train_label'])
test_acc = acc_multiclass(formulas, dataset['test_input'], dataset['test_label'])

In [77]:
test_acc

tensor(0.3750)

In [76]:
train_acc

tensor(0.4537)

In [12]:
#pred=torch.argmax(mlp_model(X), dim=1)

#sum(pred==y)/len(pred)